# Input and Target Data

This section describes the file structure of the data files containing the input data (satellite observations and ancillary data) and the target data. While this information is relevant for users who want to develop their own data loaders or evaluate external retrievals, the ``satrain`` dataset also provides ready-to-use PyTorch datasets that avoid manual data handling.

## Input Data

The SatRain dataset distinguishes a total of seven input sources. The data are stored in separate files in the format ``<prefix>_<timestamp>.nc`` with a uniqe ``prefix`` identifying the data source and a shared timestamp corresponding to the median time of the PMW overpass. The different input sources are:

 - PMW observations: ``atms`` and ``gmi``, depending on the base sensor
 - Geostationary Vis and IR: ``geo`` for the geo observations closest to the PMW overpass and ``geo_t`` for the time-resolved observations.
 - Geostationary IR: ``geo_ir`` for the Geo-IR observations closest to the PMW overpass and ``geo_ir_t`` for the time-resolved observations.
 - Ancillary data: ``ancillary``

Below, we take look the file structure of these input data.


## GMI and ATMS Observations

The GMI and ATMS observations contain the variables ``observations`` and ``earth_incidence_angle`` containing the PMW observations and the corresponding earth-incidence angles. Since the data loaded below is in gridded format, the spatial dimensions are latitude and longitude. For the on-swath data they are ``scan`` and ``pixel``.

> **Note**: The PMW files also contain the GPM L1C file and the start and end of the extracted scan range, which can be helpful to extract retrieval results from external retrieval algorithms.

In [5]:
import xarray as xr
from satrain.data import get_files
training_files = get_files("gmi", split="training", input_data=["gmi"], geometry="gridded", subset="xs")
# Load the first training file from the 'gmi' files in the training dataset.
xr.load_dataset(training_files["gmi"][0])

<xarray.Dataset> Size: 14MB
Dimensions:                (latitude: 256, longitude: 256, channel: 13)
Coordinates:
  * latitude               (latitude) float64 2kB 46.55 46.51 ... 37.31 37.27
  * longitude              (longitude) float64 2kB -77.76 -77.72 ... -68.48
Dimensions without coordinates: channel
Data variables:
    observations           (latitude, longitude, channel) float64 7MB 245.8 ....
    earth_incidence_angle  (latitude, longitude, channel) float64 7MB 52.79 ....
    scan_time              (latitude, longitude) datetime64[ns] 524kB 2018-01...
Attributes:
    satellite:       GPM
    sensor:          GMI
    preprocessor:    2407c
    scan_start:      1009
    scan_end:        1413
    gpm_input_file:  1C-R.GPM.GMI.XCAL2016-C.20180107-S191142-E204417.021938....
    lower_left_row:  147
    lower_left_col:  293
    pixel_start:     391
    pixel_end:       647

The ATMS files are only available when the ``base_sensor`` is ATMS. They contain a different number of channels.

In [6]:
import xarray as xr
from satrain.data import get_files
training_files = get_files("atms", split="training", input_data=["atms"], geometry="gridded", subset="xs")
# Load the first training file from the 'atms' files in the training dataset.
xr.load_dataset(training_files["atms"][0])

<xarray.Dataset> Size: 10MB
Dimensions:                (latitude: 256, longitude: 256, channel: 9)
Coordinates:
  * latitude               (latitude) float64 2kB 32.07 32.03 ... 22.83 22.79
  * longitude              (longitude) float64 2kB -82.92 -82.89 ... -73.65
Dimensions without coordinates: channel
Data variables:
    observations           (latitude, longitude, channel) float64 5MB nan ......
    earth_incidence_angle  (latitude, longitude, channel) float64 5MB nan ......
    scan_time              (latitude, longitude) datetime64[ns] 524kB NaT ......
Attributes:
    satellite:       NOAA20
    sensor:          ATMS
    preprocessor:    2412c
    scan_start:      2181
    scan_end:        2993
    gpm_input_file:  1C.NOAA20.ATMS.XCAL2019-V.20180107-S050836-E065004.00070...
    lower_left_row:  137
    lower_left_col:  110
    pixel_start:     36
    pixel_end:       292

## Geo Observations

The Geo Vis and IR observations contain all observations in a single variable ``observations``. Visible channels are stored using reflectance in percent and thermal IR channels use K. The ``geo_t`` files contain an additional dimension containing all 10-minute observations within an hour of the overpass.

> **Note**: Since the Vis/IR observations for different testing domains are derived from different instruments, the number of channels and their spectral coverage changes across the testing domains.

In [7]:
import xarray as xr
from satrain.data import get_files
training_files = get_files("gmi", split="training", input_data=["geo"], geometry="gridded", subset="xs")
# Load the first GEO training data file.
xr.load_dataset(training_files["geo"][0])

<xarray.Dataset> Size: 9MB
Dimensions:       (latitude: 256, longitude: 256, channel: 16)
Coordinates:
  * latitude      (latitude) float32 1kB 46.55 46.51 46.47 ... 37.34 37.31 37.27
  * longitude     (longitude) float32 1kB -77.76 -77.72 -77.69 ... -68.52 -68.48
    time          (latitude, longitude) datetime64[ns] 524kB 2018-01-07T19:43...
Dimensions without coordinates: channel
Data variables:
    observations  (latitude, longitude, channel) float64 8MB 21.9 ... 254.1

## Geo-IR Observations

The Geo IR contain globally merged observations from the IR window channels around $11 \mu m$. The multi-timestep files containg 16 half-hourly timesteps centered on the overpass time.


In [9]:
import xarray as xr
from satrain.data import get_files
training_files = get_files("gmi", split="training", input_data=["geo_ir"], geometry="gridded", subset="xs")
# Load the first Geo-IR training data file.
xr.load_dataset(training_files["geo_ir"][0])

<xarray.Dataset> Size: 1MB
Dimensions:       (latitude: 256, longitude: 256)
Coordinates:
  * latitude      (latitude) float64 2kB 46.55 46.51 46.47 ... 37.34 37.31 37.27
  * longitude     (longitude) float64 2kB -77.76 -77.72 -77.69 ... -68.52 -68.48
    time          (latitude, longitude) datetime64[ns] 524kB 2018-01-07T20:00...
Data variables:
    observations  (latitude, longitude) float64 524kB 249.0 248.0 ... 263.0
Attributes:
    BeginDate:       2018-01-07
    BeginTime:       16:00:00.000Z
    EndDate:         2018-01-07
    EndTime:         16:59:59.999Z
    FileHeader:      StartGranuleDateTime=2018-01-07T16:00:00.000Z;\nStopGran...
    InputPointer:    merg_2018010716_4km-pixel
    title:           NCEP/CPC 4km Global (60N - 60S) IR Dataset
    ProductionTime:  2018-01-09T00:19:23.000Z

## Ancillary data

The ancillary data files contain the ERA5 data, GPROF surface type, and surface elevation all in separate variables.



In [10]:
import xarray as xr
from satrain.data import get_files
training_files = get_files("gmi", split="training", input_data=["ancillary"], geometry="gridded", subset="xs")
# Load the first ancillary training data file.
xr.load_dataset(training_files["ancillary"][0])

<xarray.Dataset> Size: 6MB
Dimensions:                          (latitude: 256, longitude: 256)
Coordinates:
  * latitude                         (latitude) float32 1kB 46.55 ... 37.27
  * longitude                        (longitude) float32 1kB -77.76 ... -68.48
    time                             (latitude, longitude) datetime64[ns] 524kB ...
Data variables: (12/19)
    ten_meter_wind_u                 (latitude, longitude) float32 262kB 1.49...
    ten_meter_wind_v                 (latitude, longitude) float32 262kB 5.10...
    two_meter_dew_point              (latitude, longitude) float32 262kB 255....
    two_meter_temperature            (latitude, longitude) float32 262kB 258....
    cape                             (latitude, longitude) float32 262kB 0.46...
    sea_ice_concentration            (latitude, longitude) float32 262kB nan ...
    ...                               ...
    total_column_water_vapor         (latitude, longitude) float32 262kB 7.33...
    total_precipitation              (latitude, longitude) float32 262kB 0.00...
    convective_precipitation         (latitude, longitude) float32 262kB 0.0 ...
    leaf_area_index                  (latitude, longitude) float32 262kB 1.54...
    surface_type                     (latitude, longitude) float32 262kB 11.0...
    elevation                        (latitude, longitude) float32 262kB 308....
Attributes:
    Conventions:  CF-1.6
    history:      2021-03-16 13:19:36 GMT by grib_to_netcdf-2.16.0: /opt/ecmw...
    scan_start:   1009
    scan_end:     1413

## Target data

Finally, the target files contain the precipitation estimates as well as the radar-quality index (RQI), the gauge-correction factor (GCF), and the precipitation type fraction.


In [11]:
import xarray as xr
from satrain.data import get_files
training_files = get_files("gmi", split="training", input_data=[], geometry="gridded", subset="xs") # Target data is always included
# Load the first ancillary training data file.
xr.load_dataset(training_files["target"][0])

<xarray.Dataset> Size: 5MB
Dimensions:                  (latitude: 256, longitude: 256)
Coordinates:
  * latitude                 (latitude) float64 2kB 46.55 46.51 ... 37.31 37.27
  * longitude                (longitude) float64 2kB -77.76 -77.72 ... -68.48
Data variables:
    surface_precip           (latitude, longitude) float64 524kB 0.91 ... nan
    radar_quality_index      (latitude, longitude) float64 524kB 0.0 0.0 ... nan
    gauge_correction_factor  (latitude, longitude) float32 262kB 0.9156 ... nan
    valid_fraction           (latitude, longitude) float64 524kB 1.0 1.0 ... 0.0
    precip_fraction          (latitude, longitude) float64 524kB 1.0 1.0 ... nan
    snow_fraction            (latitude, longitude) float64 524kB 1.0 1.0 ... nan
    hail_fraction            (latitude, longitude) float64 524kB 0.0 0.0 ... nan
    convective_fraction      (latitude, longitude) float64 524kB 0.0 0.0 ... nan
    stratiform_fraction      (latitude, longitude) float64 524kB 0.0 0.0 ... nan
    time                     (latitude, longitude) datetime64[ns] 524kB 2018-...
Attributes:
    lower_left_col:  2419
    lower_left_row:  1046